In [2]:
import numpy as np
import pandas as pd
import datetime as dt

import requests
import xlrd
import csv

import matplotlib.pyplot as plt
from IPython.display import clear_output
try:
  import mplcyberpunk
except:
  !pip install mplcyberpunk
  clear_output()
  import mplcyberpunk

plt.style.use("cyberpunk")


import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipywidgets.embed import embed_minimal_html

try:
  import nbinteract as nbi
except:
  !pip install nbinteract
  clear_output()
  import nbinteract as nbi

import os

In [3]:
def csv_from_excel(filename,sheetname):
    wb = xlrd.open_workbook(filename+'.xlsx')
    sh = wb.sheet_by_name(sheetname)
    your_csv_file = open(filename+'.csv', 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()

In [4]:
dailycases = requests.get('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

if dailycases.status_code == 200:
    with open('dailycases.csv', 'wb') as f:
        f.write(dailycases.content)

In [5]:
testingdata = requests.get('https://www.ecdc.europa.eu/sites/default/files/documents/weekly_testing_data_EUEEAUK_2020-09-16.xlsx')

if testingdata.status_code == 200:
    with open('testingdata.xlsx', 'wb') as f:
        f.write(testingdata.content)

wb = xlrd.open_workbook('testingdata.xlsx')
print(wb.sheet_names())

csv_from_excel('testingdata','Sheet1')

['Sheet1']


In [6]:
dog = dt.date.today() - dt.timedelta(days=1)

In [7]:
ndays = 0

while not pd.isna(ndays):
  date = dt.date.today() - ndays*dt.timedelta(days=1)

  hospitaldata = requests.get('https://www.ecdc.europa.eu/sites/default/files/documents/hosp_icu_all_data_'+str(date)+'.xlsx')

  if hospitaldata.status_code == 200:
      ndays = np.nan
      with open('hospitaldata.xlsx', 'wb') as f:
          f.write(hospitaldata.content)
  else:
    ndays = ndays+1

wb = xlrd.open_workbook('hospitaldata.xlsx')
print(wb.sheet_names())

csv_from_excel('hospitaldata','Sheet1')

['Sheet1']


In [8]:
year = dt.date.today().isocalendar()[0]
week = dt.date.today().isocalendar()[1]

while not np.isnan(week):
  deathdata = requests.get('https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/deathregistrationsandoccurrencesbylocalauthorityandhealthboard/'+str(year)+'/lahbtablesweek'+str(week)+'.xlsx')
  
  if deathdata.status_code == 200:
      week = np.nan
      with open('deathdata.xlsx', 'wb') as f:
          f.write(deathdata.content)
  else:
    print(week)
    week = week - 1
wb = xlrd.open_workbook('deathdata.xlsx')
print(wb.sheet_names())

csv_from_excel('deathdata','Registrations - All data')

38
37
['Contents', 'Information', 'Terms and conditions', 'Registrations - All data', 'Registrations - Pivot table', 'Occurrences - All data', 'Occurrences - Pivot table']


In [9]:
cases = pd.read_csv('dailycases.csv')
tests = pd.read_csv('testingdata.csv')
hospital = pd.read_csv('hospitaldata.csv')
death = pd.read_csv('deathdata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
death.columns = death.iloc[2].values
death = death[3:]

In [11]:
cases.columns, tests.columns, hospital.columns, death.columns

(Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths',
        'countriesAndTerritories', 'geoId', 'countryterritoryCode',
        'popData2019', 'continentExp',
        'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],
       dtype='object'),
 Index(['country', 'country_code', 'year_week', 'new_cases', 'tests_done',
        'population', 'testing_rate', 'positivity_rate', 'testing_data_source'],
       dtype='object'),
 Index(['country', 'indicator', 'date', 'year_week', 'value', 'source', 'url'], dtype='object'),
 Index(['Area code', 'Geography type', 'Area name ', 'Cause of death',
        'Week number', 'Place of death', 'Number of deaths'],
       dtype='object'))

In [12]:
cases.index = [dt.datetime(year,month,day) for year,month,day
               in zip(cases.year.values,cases.month.values,cases.day.values)]

week = dt.timedelta(days=7)
tests.index = [dt.datetime(2020,1,6)+week*(int(i[-2:])-2)
               if int(i[-2:]) != 1 else dt.datetime(2020,1,1)
               for i in tests.year_week]

hospital['datetime'] = [dt.datetime.strptime(i,'%Y-%m-%d') if not pd.isna(i) else
                        dt.datetime(2020,1,6)+week*(int(j[-2:])-2)
                        if int(j[-2:]) != 1 else dt.datetime(2020,1,1)
                        for i,j in zip(hospital.date,hospital.year_week)]

death.index = [dt.datetime(2020,1,6)+week*(int(float(i))-2)
               if int(float(i)) != 1 else dt.datetime(2020,1,1)
               for i in death['Week number'].values]

In [13]:
C1 = widgets.Dropdown(options = cases.countriesAndTerritories.unique())
C2 = widgets.Dropdown(options = hospital.country.unique())
In1 = widgets.Dropdown(options = hospital.indicator.unique())


def intplot(Country1,Country2,Indicator):

  ccases = cases[cases.countriesAndTerritories == Country1]
  chosp = hospital[hospital.country == Country2]

  plt.figure(figsize=(8,13.5))
  ax1,ax2,ax3 = plt.subplot(311),plt.subplot(312),plt.subplot(313)

  ax1.plot(ccases.index,ccases.cases)
  ax2.plot(ccases.index,ccases.deaths)
  ax3.plot(chosp[chosp.indicator==Indicator].datetime,
          chosp[chosp.indicator==Indicator].value)

  ax1.set_title(Country1+': Daily Cases')
  ax2.set_title(Country1+': Daily Deaths')
  ax3.set_title(Country2+': '+Indicator)
  plt.show()

In [14]:
interact(intplot,Country1=C1,Country2=C2,Indicator=In1)

interactive(children=(Dropdown(description='Country1', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra'…

<function __main__.intplot>

In [16]:
embed_minimal_html('interactive_plot.html', views=None, title='Widgets export')

In [28]:
try:
  import nbinteract as nbi
except:
  !pip install nbinteract
  clear_output()
  import nbinteract as nbi

import os

'/content'

In [31]:
nbi.publish('tbloch1/website/master', '/content/covid.ipynb')

ValueError: ignored